I break the notebook into separate steps. 
* [Step 1](#step1): Import Datasets
* [Step 2](#step2): Create ensembled features using Resnet50, Xception, and InceptionV3
* [Step 3](#step3): Create a CNN to Classify Dog Breeds (using Transfer Learning)

<a id='step1'></a>
## Step 1: Import Datasets

### Import Dog Dataset

In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

Using TensorFlow backend.


There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


In [141]:
train_feature.shape[1:]

(6144,)

<a id='step2'></a>
## Step 2: Create ensembled features using Resnet50, Xception, and InceptionV3

In [67]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

from keras.callbacks import ModelCheckpoint 

In [120]:
# features created by the deep convolutional layers of Resnet50 
bottleneck_features = np.load('bottleneck_features/DogResnet50Data.npz')
train_Resnet50 = bottleneck_features['train']
valid_Resnet50 = bottleneck_features['valid']
test_Resnet50 = bottleneck_features['test']

In [121]:
# features created by the deep convolutional layers of InceptionV3
bottleneck_features = np.load('bottleneck_features/DogInceptionV3Data.npz')
train_InceptionV3 = bottleneck_features['train']
valid_InceptionV3 = bottleneck_features['valid']
test_InceptionV3 = bottleneck_features['test']

In [122]:
# features created by the deep convolutional layers of Xception
bottleneck_features = np.load('bottleneck_features/DogXceptionData.npz')
train_Xception = bottleneck_features['train']
valid_Xception = bottleneck_features['valid']
test_Xception = bottleneck_features['test']

In [123]:
# transfer 4d matrix to 2d
InceptionV3_flatten_model = Sequential()
InceptionV3_flatten_model.add(GlobalAveragePooling2D(input_shape=train_InceptionV3.shape[1:]))
InceptionV3_flatten_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_13  (None, 2048)              0         
Total params: 0.0
Trainable params: 0.0
Non-trainable params: 0.0
_________________________________________________________________


In [124]:
# transfer 4d matrix to 2d
Xception_flatten_model = Sequential()
Xception_flatten_model.add(GlobalAveragePooling2D(input_shape=train_Xception.shape[1:]))
Xception_flatten_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_14  (None, 2048)              0         
Total params: 0.0
Trainable params: 0.0
Non-trainable params: 0.0
_________________________________________________________________


In [125]:
# transfer 4d matrix to 2d
Resnet50_flatten_model = Sequential()
Resnet50_flatten_model.add(GlobalAveragePooling2D(input_shape=train_Resnet50.shape[1:]))
Resnet50_flatten_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_15  (None, 2048)              0         
Total params: 0.0
Trainable params: 0.0
Non-trainable params: 0.0
_________________________________________________________________


In [126]:
# transfer 4d matrix to 2d
train_InceptionV3 = InceptionV3_flatten_model.predict(train_InceptionV3)
valid_InceptionV3 = InceptionV3_flatten_model.predict(valid_InceptionV3)
test_InceptionV3 = InceptionV3_flatten_model.predict(test_InceptionV3)

train_Xception = Xception_flatten_model.predict(train_Xception)
valid_Xception = Xception_flatten_model.predict(valid_Xception)
test_Xception = Xception_flatten_model.predict(test_Xception)

train_Resnet50 = Resnet50_flatten_model.predict(train_Resnet50)
valid_Resnet50 = Resnet50_flatten_model.predict(valid_Resnet50)
test_Resnet50 = Resnet50_flatten_model.predict(test_Resnet50)

In [127]:
train_feature = np.concatenate((train_InceptionV3, train_Xception, train_Resnet50), axis=1)
valid_feature = np.concatenate((valid_InceptionV3, valid_Xception, valid_Resnet50), axis=1)
test_feature = np.concatenate((test_InceptionV3, test_Xception, test_Resnet50), axis=1)

In [128]:
test_feature.shape[1:]

(6144,)

<a id='step3'></a>
## Step 3: Create a CNN to Classify Dog Breeds (using Transfer Learning)

In [135]:
# dense layer model using the ensembled features from three DCN
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=train_feature.shape[1:]))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(133, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 1024)              6292480   
_________________________________________________________________
dropout_24 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_34 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_25 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 133)               68229     
Total params: 6,885,509.0
Trainable params: 6,885,509.0
Non-trainable params: 0.0
_________________________________________________________________


In [136]:
### Compile the model.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [137]:
### Train the model and save the prameters of the model with the best performance in validation set
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.model1.hdf5', monitor='val_acc',
                               verbose=1, save_best_only=True)

model.fit(train_feature, train_targets, 
          validation_data=(valid_feature, valid_targets),
          epochs=20, batch_size=400, callbacks=[checkpointer], verbose=2)

Train on 6680 samples, validate on 835 samples
Epoch 1/20
Epoch 00000: val_acc improved from -inf to 0.79521, saving model to saved_models/weights.best.model1.hdf5
15s - loss: 2.3075 - acc: 0.4754 - val_loss: 0.6509 - val_acc: 0.7952
Epoch 2/20
Epoch 00001: val_acc improved from 0.79521 to 0.85749, saving model to saved_models/weights.best.model1.hdf5
6s - loss: 0.6304 - acc: 0.8129 - val_loss: 0.4886 - val_acc: 0.8575
Epoch 3/20
Epoch 00002: val_acc improved from 0.85749 to 0.88623, saving model to saved_models/weights.best.model1.hdf5
6s - loss: 0.4019 - acc: 0.8702 - val_loss: 0.4057 - val_acc: 0.8862
Epoch 4/20
Epoch 00003: val_acc did not improve
6s - loss: 0.2990 - acc: 0.9058 - val_loss: 0.3922 - val_acc: 0.8743
Epoch 5/20
Epoch 00004: val_acc did not improve
6s - loss: 0.2264 - acc: 0.9229 - val_loss: 0.4442 - val_acc: 0.8731
Epoch 6/20
Epoch 00005: val_acc did not improve
6s - loss: 0.1772 - acc: 0.9448 - val_loss: 0.4057 - val_acc: 0.8814
Epoch 7/20
Epoch 00006: val_acc did n

In [139]:
model.load_weights('saved_models/weights.best.model1.hdf5')

In [140]:
### Calculate classification accuracy on the test dataset.
predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in test_feature]

# report test accuracy
test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(test_targets, axis=1))/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 88.8756%


In [113]:
from keras.preprocessing import image  
import urllib.request
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)


import os
 
# METHOD #1: OpenCV, NumPy, and urllib
def url_to_tensor(url):
    # download the image, convert it to a NumPy array, and then read
    urllib.request.urlretrieve(url, 'img_buf.jpg')
    tensor = path_to_tensor('img_buf.jpg')
    os.remove('img_buf.jpg')
    # return the image
    return tensor

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True    

In [114]:
URL = 'https://i.amz.mshcdn.com/K47URRdccsvcDhyc1IpfrQ0WN9s=/1200x630/2017%2F08%2F11%2Ff2%2Ffda8c2c1913a45bab33a5e04b897cb9b.4021f.jpg'
img = url_to_tensor(URL)

In [22]:
from extract_bottleneck_features import *

def model_predict_breed(img_path):
    # extract bottleneck features
    Resnet50_feature = extract_Resnet50(path_to_tensor(img_path))
    InceptionV3_feature = extract_InceptionV3(path_to_tensor(img_path))
    Xception_feature = extract_Xception(path_to_tensor(img_path))
    
    InceptionV3_feature = InceptionV3_flatten_model.predict(InceptionV3_feature)
    Xception_feature = Xception_flatten_model.predict(Xception_feature)
    Resnet50_feature = Resnet50_flatten_model.predict(Resnet50_feature)

    feature = np.concatenate((InceptionV3_feature, Xception_feature, Resnet50_feature), axis=1)
    # obtain predicted vector
    predicted_vector = model.predict(feature)
    # return dog breed that is predicted by the model
    return dog_names[np.argmax(predicted_vector)]

In [26]:
model_predict_breed('test\\damon.jpg')

'Havanese'

In [27]:
model_predict_breed('test\\gumu.jpg')

'Old_english_sheepdog'